In [1]:
import tensorflow as tf
from lib.utils import model ,preproc, newBuild, newGan
from keras.preprocessing.image import ImageDataGenerator

# Experiment on flower paintings

In [2]:
res = 128
channel = 3
# Preprocessing data with custom function
batch_size = 4
img_shape = (res,res,channel)
epochs = 2000
latent_dim = 128
n_images = 16
image_dir = "../../oDig/flowers/"
checkpoint_dir = "../../checkpoints/flower_check/"
#data_loc = "../../datasets/celeb/"
image_freq = 200
s = tf.random.normal([n_images, latent_dim]),
checkpoint_freq = 200

2022-04-26 15:59:06.510486: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 15:59:06.515823: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 15:59:06.516171: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 15:59:06.541143: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
datagen = ImageDataGenerator(preprocessing_function=preproc.tanh_preprocess,
                             horizontal_flip=True,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.1,
                             rotation_range=5,
                             fill_mode='nearest'
                             )
train_dir = '/home/sandor/Dev/final/finalproject/gan/datasets/flowers'

dataset = datagen.flow_from_directory(
     train_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode=None
)

Found 969 images belonging to 1 classes.


In [4]:
constants = dict(
    num_conv_layers = 4,
    img_shape = img_shape,
    dim = 8,
    output = channel,
    latent_dim = latent_dim,
    kernel = 4,
    stride = 2
)
callback = dict(
    num_img = n_images,
    cFreq = checkpoint_freq,
    iFreq = image_freq,
    seed = s,
    loc = image_dir
)
gan_attributes = dict(
    check_location= checkpoint_dir,
    b_size=  batch_size,
    l_dim= latent_dim,
    data = dataset,
    epochs=epochs,
)

In [5]:
name = 'Model_1-Same-Tanh-Batch-mDrop'
parameters = dict(
    model_name=name,
    filter_mode=3,
    gen_filter=7,
    disc_filter=6,
    is_batchnorm=True,
    is_multi_drop=True,
    is_tanh=True
)
model1 = model.ModelSettings(**constants, **parameters)
gan1 = newBuild.GANBuilder(atrib=model1, name=name, **gan_attributes)
g = newBuild.make_gan(gan1)

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense (Dense)               (None, 8192)              1056768   
                                                                 
 batch_normalization_4 (Batc  (None, 8192)             32768     
 hNormalization)                                                 
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 8192)              0         
                                                                 
 reshape (Reshape)           (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 16, 16, 128)      262272    
 nspose)                                                 

In [6]:
g.fit(dataset, epochs=epochs, callbacks = [newGan.GANCallback(name= name,**callback)])

ImportError: Image transformations require SciPy. Install SciPy.